In [1]:
!git clone https://github.com/sjoshi804/sas-data-efficient-contrastive-learning.git

Cloning into 'sas-data-efficient-contrastive-learning'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 272 (delta 36), reused 39 (delta 29), pack-reused 220
Receiving objects: 100% (272/272), 24.19 MiB | 7.56 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [2]:
%cd ./sas-data-efficient-contrastive-learning

/content/sas-data-efficient-contrastive-learning


In [ ]:
# asposestorage==1.0.2
# openai-clip
# fast_pytorch_kmeans==0.1.6
# numpy==1.23.1
# pandas==1.5.1
# Pillow==9.5.0
# setuptools==63.4.1
# torch==1.12.1
# torchvision==0.13.1
# tqdm==4.64.1
# wandb

# Remember to update the requirements.txt file before running the next line

## Replace sas-data-efficient-contrastive-learning/requirements.txt with the block above (uncomment first)!!

OR (individually) replace clip in requirements with openai-clip, delete Pillow==9.2.0, add wandb.

In [3]:
# Replace clip in requirements with openai-clip, delete Pillow, add wandb, or
# copy from the block above (uncomment first)!!
!pip install -r requirements.txt

!pip install sas-pip/

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.9 MB/s eta 0:

Processing ./sas-pip
  Preparing metadata (setup.py) ... done
  Created wheel for sas: filename=sas-1.0-py3-none-any.whl size=6289 sha256=65bb5483a65d0b0967326351028a27d422c1cee093a4c46612ffbfa8464e4d33
  Stored in directory: /root/.cache/pip/wheels/5a/80/5f/4f77ca879b91a1f5f78a11be811243d69527d564efb1eb9d6b
Successfully built sas


## The next block does most of the work.

It creates the noisy dataset and then partitions it. The following block uses the result to run SAS, then we can determine the percentage of noisy examples removed.

In [ ]:

import random
import os
import torch

import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter

from sas.approx_latent_classes import clip_approx
from sas.subset_dataset import SASSubsetDataset

cifar100 = torchvision.datasets.CIFAR100(
    "/data/cifar100/", transform=transforms.ToTensor(), download=True)
device = "cuda:0"

# Choose noisy indices.
percent_noisy = .1
noisy_indices = random.sample(range(1, len(cifar100)), int(percent_noisy * len(cifar100)))

x = cifar100[0][0]
a, b, c = x.shape

def add_noise_to_image(x):
  a, b, c = x.shape
  d = torch.randn(a, b, c) * .3
  return x + d

noise32 = {}
noise224 = {}

# TODO: Why 224?
for i in noisy_indices:
  noise32[i] = torch.randn(3, 32, 32) * .3
  noise224[i] = torch.randn(3, 224, 224) * .3

# Overwrite the CIFAR dataset with my own which replaces specific, pre-chosen
# samples with noise.
class MyDataset(torchvision.datasets.CIFAR100):
    def __getitem__(self, index):
        image, label = super().__getitem__(index)
        # print(image.shape)
        if index in noisy_indices:
          # return torch.zeros_like(image), label
          if image.shape[1] == 32:
            return image + noise32[i], label
          elif image.shape[1] == 32:
            return image + noise224[i], label
          else:
            print('BEN')
            print(image.shape)
        else:
          return image, label

# Create dataset.
cifar_noisy = MyDataset(
    "/data/cifar100/", transform=transforms.ToTensor(), download=True)

rand_labeled_examples_indices = random.sample(range(len(cifar_noisy)), 500)
rand_labeled_examples_labels = [
    cifar_noisy[i][1] for i in rand_labeled_examples_indices]

partition = clip_approx(
    img_trainset=cifar_noisy,
    labeled_example_indices=rand_labeled_examples_indices,
    labeled_examples_labels=rand_labeled_examples_labels,
    num_classes=100,
    device=device
)

Files already downloaded and verified


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7aa71f10c5e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    finally:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    self._shutdown = True
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 


Files already downloaded and verified
BEN
torch.Size([3, 224, 224])
BEN
BENtorch.Size([3, 224, 224])BEN

BEN

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])BEN

torch.Size([3, 224, 224])torch.Size([3, 224, 224])

BENBENBENBENBEN
torch.Size([3, 224, 224])BEN





torch.Size([3, 224, 224])torch.Size([3, 224, 224])torch.Size([3, 224, 224])


BENtorch.Size([3, 224, 224])torch.Size([3, 224, 224])

BEN
torch.Size([3, 224, 224])

BENBENBENtorch.Size([3, 224, 224])



torch.Size([3, 224, 224])torch.Size([3, 224, 224])torch.Size([3, 224, 224])
BENBEN
BEN
torch.Size([3, 224, 224])


BEN
torch.Size([3, 224, 224])BENtorch.Size([3, 224, 224])



torch.Size([3, 224, 224])torch.Size([3, 224, 224])

BENBENBEN
torch.Size([3, 224, 224])


torch.Size([3, 224, 224])torch.Size([3, 224, 224])
BEN
BENBEN


BENtorch.Size([3, 224, 224])
torch.Size([3, 224, 224])
BEN
BEN
torch.Size([3, 224, 224])torch.Size([3, 224, 224])

torch.Size([3, 224, 224])

torch.Size([3, 224, 224])BEN

BENtorch.Size([3, 224, 224])

TypeError: ignored


BENtorch.Size([3, 224, 224])

torch.Size([3, 224, 224])
BEN
BENBEN
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
BENBEN


torch.Size([3, 224, 224])
torch.Size([3, 224, 224])

## Run SAS and then see how well it did

In [7]:
import torch
from torch import nn

class ProxyModel(nn.Module):
    def __init__(self, net, critic):
        super().__init__()
        self.net = net
        self.critic = critic
    def forward(self, x):
        return self.critic.project(self.net(x))

# Load the proxy model.
net = torch.load("proxy-cifar100-resnet10-399-net.pt")
critic = torch.load("proxy-cifar100-resnet10-399-critic.pt")
proxy_model = ProxyModel(net, critic)


# Here we actually run SAS.
subset_fraction = 0.2

# Get Subset using SAS
subset_dataset = SASSubsetDataset(
    dataset=cifar_noisy,
    subset_fraction=subset_fraction,
    num_downstream_classes=100,
    device=device,
    proxy_model=proxy_model,
    approx_latent_class_partition=partition,
    verbose=True
)

In [ ]:
# This block calculates the success at removing the noisy examples.

print(len(cifar_noisy))
print(len(subset_dataset))
print(len(noisy_indices))

intersection = set(subset_dataset.subset_indices).intersection(
    set(noisy_indices))

print(len(intersection))

print('SAS subset fraction: ' + str(subset_fraction))
print('Fraction of examples removed: ' + str(1 - subset_fraction))

print('Fraction of noisy examples remaining (should be close to 0):')
print(len(intersection) / len(noisy_indices))

print('Fraction of noisy examples removed (should be close to 1,' +
      'definitely greater than 1-|subset_fraction|):')
print(1 - len(intersection) / len(noisy_indices))

50000
10000
5000
1085
SAS subset fraction: 0.2
Fraction of examples removed: 0.8
Fraction of noisy examples remaining (should be close to 0):
0.217
Fraction of noisy examples removed (should be close to 1,definitely greater than 1-|subset_fraction|):
0.783


In [5]:
!python simclr.py --arch resnet10 --dataset cifar10 --checkpoint-freq 10

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "Don't visualize my results"
wandb: Tracking run with wandb version 0.16.1
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
==> Preparing data..
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:891: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
100% 170498071/170498071 [00:13<00:00, 12939749.94it/s]
Extracting /data/cifar10/cifar-10-python.tar.gz to /data/cifar10/
Files already downloaded and verified
Files already downloaded and verified
subset_size: 50000
==> Building model..
step: 0
Loss: 6.040 : 100% 98/98 [01:19<00:00,  1.24it/s]
train_loss: 6.039570939784148
step: 1
Loss: 5.851 : 100% 98/98 [01:25<00:00,  1.

In [8]:
net = torch.load("2023-12-1504:18:53.690899-cifar10-resnet10-9-net.pt")
critic = torch.load("2023-12-1504:18:53.690899-cifar10-resnet10-9-critic.pt")
proxy_model = ProxyModel(net, critic)

In [20]:
import random
import os
import torch

import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter

from sas.approx_latent_classes import clip_approx
from sas.subset_dataset import SASSubsetDataset

cifar10 = torchvision.datasets.CIFAR10(
    "/data/cifar10/", transform=transforms.ToTensor(), download=True)

x1 = cifar10[0][0].to('cuda')
x2 = cifar10[1][0].to('cuda')
y = torch.stack((x1, x2))
print(y.shape)

a = proxy_model(y)
print(a.shape)

Files already downloaded and verified
torch.Size([2, 3, 32, 32])
torch.Size([2, 128])


In [22]:
!tar -xvf CIFAR-10-C.tar
!ls CIFAR-10-C

CIFAR-10-C/
CIFAR-10-C/fog.npy
CIFAR-10-C/jpeg_compression.npy
CIFAR-10-C/zoom_blur.npy
CIFAR-10-C/speckle_noise.npy
CIFAR-10-C/glass_blur.npy
CIFAR-10-C/spatter.npy
CIFAR-10-C/shot_noise.npy
CIFAR-10-C/defocus_blur.npy
CIFAR-10-C/elastic_transform.npy
CIFAR-10-C/gaussian_blur.npy
CIFAR-10-C/frost.npy
CIFAR-10-C/saturate.npy
CIFAR-10-C/brightness.npy
CIFAR-10-C/snow.npy
CIFAR-10-C/gaussian_noise.npy
CIFAR-10-C/motion_blur.npy
CIFAR-10-C/contrast.npy
CIFAR-10-C/impulse_noise.npy
CIFAR-10-C/labels.npy
CIFAR-10-C/pixelate.npy
brightness.npy	       fog.npy		   glass_blur.npy	 motion_blur.npy  snow.npy
contrast.npy	       frost.npy	   impulse_noise.npy	 pixelate.npy	  spatter.npy
defocus_blur.npy       gaussian_blur.npy   jpeg_compression.npy  saturate.npy	  speckle_noise.npy
elastic_transform.npy  gaussian_noise.npy  labels.npy		 shot_noise.npy   zoom_blur.npy


In [24]:
# Unpack CIFAR-10-C

import numpy as np
import matplotlib.pyplot as plt
import os

DIR = 'CIFAR-10-C'
lb = np.load(os.path.join(DIR,'labels.npy'))
cifar10c = np.load('CIFAR-10-C/fog.npy')
print(lb)

cifar10c.shape

[3 8 8 ... 5 1 7]


(50000, 32, 32, 3)

In [82]:
# Copied from sas-pip/sas/approx_latent_classes.py for debugging.

from copy import deepcopy
from typing import List

import clip
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from fast_pytorch_kmeans import KMeans
from tqdm import tqdm

def clip_approx(
    img_trainset: torch.utils.data.Dataset,
    labeled_example_indices: List[int],
    labeled_examples_labels: np.array,
    num_classes: int,
    device: torch.device,
    batch_size: int = 512,
    verbose: bool = False,
):
    Z = encode_using_clip(
        img_trainset=img_trainset,
        device=device,
        batch_size=batch_size,
        verbose=verbose
    )
    clf = train_linear_classifier(
        X=Z[labeled_example_indices],
        y=torch.tensor(labeled_examples_labels),
        representation_dim=len(Z[0]),
        num_classes=num_classes,
        device=device,
        verbose=verbose
    )
    preds = []
    for start_idx in range(0, len(Z), batch_size):
        preds.append(torch.argmax(clf(Z[start_idx:start_idx + batch_size]).detach(), dim=1).cpu())
    preds = torch.cat(preds).numpy()

    return partition_from_preds(preds)

def clip_0shot_approx(
    img_trainset: torch.utils.data.Dataset,
    class_names: List[str],
    device: torch.device,
    verbose: bool = False,
):
    model, preprocess = clip.load("ViT-B/32", device=device)
    img_trainset = deepcopy(img_trainset)
    img_trainset.transform = preprocess

    zeroshot_weights = zeroshot_classifier(
        class_names=class_names,
        device=device,
        verbose=verbose
    )
    logits = []
    loader = torch.utils.data.DataLoader(img_trainset, batch_size=32, num_workers=2)
    with torch.no_grad():
        for input in tqdm(loader, "0-shot classification using provided text names for classes", disable=not verbose):
            # predict
            image_features = model.encode_image(input[0].to(device=device))
            image_features /= image_features.norm(dim=-1, keepdim=True)
            logits.append(100. * image_features @ zeroshot_weights)

    preds = []
    for logit in logits:
        preds.append(logit.topk(1, 1, True, True)[1].t()[0])

    return partition_from_preds(preds)

def kmeans_approx(
    trainset: torch.utils.data.Dataset,
    proxy_model: nn.Module,
    num_classes: int,
    device: torch.device,
    verbose: bool = False
):
    proxy_model.eval()
    Z = []
    with torch.no_grad():
        loader = torch.utils.data.DataLoader(trainset, batch_size=32, num_workers=2)
        for input in tqdm(loader, "Encoding data using proxy model provided", disable=not verbose):
            Z.append(proxy_model(input[0].to(device)))
    Z = torch.cat(Z, dim=0).to("cpu")
    kmeans = KMeans(n_clusters=num_classes, mode='euclidean', verbose=int(verbose), max_iter=1000)
    preds = kmeans.fit_predict(Z).cpu().numpy()
    return partition_from_preds(preds)

def encode_using_clip(
        img_trainset: torch.utils.data.Dataset,
        device: torch.device,
        batch_size=512,
        verbose: bool = False,
):
    model, preprocess = clip.load("ViT-B/32", device=device)
    img_trainset = deepcopy(img_trainset)
    img_trainset.transform = preprocess

    loader = torch.utils.data.DataLoader(img_trainset, batch_size=batch_size, num_workers=8)
    Z = []
    with torch.no_grad():
        for input in tqdm(loader, desc="Encoding images using CLIP", disable=not verbose):
            Z.append(model.encode_image(input[0].to(device)))
    Z = torch.cat(Z, dim=0).to(torch.float32)
    return Z

def partition_from_preds(preds):
    partition = {}
    for i, pred in enumerate(preds):
        if pred not in partition:
            partition[pred] = []
        partition[pred].append(i)
    return partition

def train_linear_classifier(
    X: torch.tensor,
    y: torch.tensor,
    representation_dim: int,
    num_classes: int,
    device: torch.device,
    reg_weight: float = 1e-3,
    n_lbfgs_steps: int = 500,
    verbose=False,
):
    if verbose:
        print('\nL2 Regularization weight: %g' % reg_weight)

    criterion = nn.CrossEntropyLoss()
    X_gpu = X.to(device)
    y_gpu = y.to(device)

    # Should be reset after each epoch for a completely independent evaluation
    clf = nn.Linear(representation_dim, num_classes).to(device)
    clf_optimizer = optim.LBFGS(clf.parameters())
    clf.train()

    for _ in tqdm(range(n_lbfgs_steps), desc="Training linear classifier using fraction of labels", disable=not verbose):
        def closure():
            clf_optimizer.zero_grad()
            raw_scores = clf(X_gpu)
            loss = criterion(raw_scores, y_gpu)
            loss += reg_weight * clf.weight.pow(2).sum()
            loss.backward()
            return loss
        clf_optimizer.step(closure)
    return clf

def zeroshot_classifier(
    class_names: List[str],
    device: torch.device,
    verbose: bool = False
):
    templates = [
        'itap of the {}.',
        'a bad photo of the {}',
        'a origami {}.',
        'a photo of the large {}.',
        'a {} in a video game.',
        'art of the {}.',
        'a photo of the small {}.',
    ]

    model, _ = clip.load("ViT-B/32")
    model = model.to(device)

    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(class_names, desc="Creating zero shot classifier", disable=not verbose):
            texts = [template.format(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).to(device) #tokenize
            class_embeddings = model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1)
    return zeroshot_weights

In [89]:
import random
import os
import torch

import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter
import torch.nn.functional as F

# from sas.approx_latent_classes import clip_approx
# from sas.subset_dataset import SASSubsetDataset

cifar10 = torchvision.datasets.CIFAR10(
    "/data/cifar10/", transform=transforms.ToTensor(), download=True, train=False)
device = "cuda:0"


def plt2(i, i2):
  """Display two images."""
  fig = plt.figure(figsize=(10, 7))

  # setting values to rows and column variables
  rows = 2
  columns = 1

  fig.add_subplot(rows, columns, 1)

  plt.imshow(i)
  fig.add_subplot(rows, columns, 2)

  plt.imshow(i2)

# x = cifar10[0][0]
# a, b, c = x.shape

# def add_noise_to_image(x):
#   a, b, c = x.shape
#   d = torch.randn(a, b, c) * .3
#   return x + d

# noise32 = {}
# noise224 = {}

# # TODO: Why 224?
# for i in noisy_indices:
#   noise32[i] = torch.randn(3, 32, 32) * .3
#   noise224[i] = torch.randn(3, 224, 224) * .3

first = True

severity = 1

noise_types = ['fog']

# for _ in range(1):
for noise_type in noise_types:
  print(noise_type)
  fname = noise_type + '.npy'
  DIR = 'CIFAR-10-C'
  cifar10c = np.load(os.path.join(DIR, fname))
  # cifar10c = np.load('CIFAR-10-C/fog.npy')

  for _ in range(1):
  # for severity in range(1,6):
    # Choose noisy indices.
    percent_noisy = .1
    noisy_indices = random.sample(range(1, len(cifar10)), int(percent_noisy * len(cifar10)))

    # Overwrite the CIFAR dataset with my own which replaces specific, pre-chosen
    # samples with noise.
    class MyDataset(torchvision.datasets.CIFAR10):
        def __getitem__(self, index):
            global first
            image, label = super().__getitem__(index)
            # print(image.shape)
            preprocess = transforms.Compose([
              transforms.Resize(256),
              transforms.CenterCrop(224),
              # transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            if index in noisy_indices:
              i = (severity - 1) * 10000 + index
              # print('BEN')
              # print(i)
              # print(index)
              # print(image.shape)
              #  torch.nn.functionnal.interpolate.
              # trans1 = transforms.ToTensor()
              # trans1 = transforms.Compose([transforms.ToTensor(), transforms.Resize(size=(224, 224)),
              #                              transforms.Normalize((0.5,), (0.5,))])
              # trans1 = transforms.Compose([transforms.ToTensor(), transforms.Resize(size=(224, 224)),
              #                             transforms.Normalize((0.5,), (0.5,))])

              preprocess2 = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
              ])

              # a = trans1(cifar10c[i])
              # a = preprocess(cifar10c[i])
              # print(a.shape)
              # a = a.permute(2, 0, 1)
              # print(a.shape)

              # out2 = F.interpolate(a.unsqueeze(dim=0), size=(224, 224), mode='bilinear')

              # out2 = out2.squeeze()
              # print(out2.shape)
              # print(out2)

              # a = torch.from_numpy(cifar10c[i])
              a = preprocess2(Image.fromarray(cifar10c[i]))
              b = torch.zeros_like(a)
              c = b + a
              # print(a.shape)
              # a = a.permute(2, 0, 1)
              # a = a / 255.0
              # print(a.shape)
              if first and image.shape[1] == 224:
                # print('BEN')
                first = False
                # plt2(image.permute(1, 2, 0), c.permute(1, 2, 0))
                # print(i)
                # print(c)
                # print(c.shape)
                # print(image)
                # print(image.shape)
                # assert False
              return c, label
              # return image, label
            else:
              return preprocess(image), label

    # Create dataset.
    cifar_noisy = MyDataset(
        "/data/cifar10/", transform=transforms.ToTensor(), download=True, train=False)

    rand_labeled_examples_indices = random.sample(range(len(cifar_noisy)), 500)
    rand_labeled_examples_labels = [
        cifar_noisy[i][1] for i in rand_labeled_examples_indices]

    partition = clip_approx(
        img_trainset=cifar_noisy,
        labeled_example_indices=rand_labeled_examples_indices,
        labeled_examples_labels=rand_labeled_examples_labels,
        num_classes=100,
        device=device
    )

    subset_fraction = .2

    # Get Subset using SAS
    subset_dataset = SASSubsetDataset(
        dataset=cifar_noisy,
        subset_fraction=subset_fraction,
        num_downstream_classes=100,
        device=device,
        proxy_model=proxy_model,
        approx_latent_class_partition=partition,
        verbose=True
    )

    print(len(cifar_noisy))

    print(len(subset_dataset))
    # print(subset_dataset.subset_indices)

    print(len(noisy_indices))

    inter = set(subset_dataset.subset_indices).intersection(set(noisy_indices))

    print(len(inter))

    print(noise_type)
    print('SAS subset fraction: ' + str(subset_fraction))

    print(len(inter) / len(noisy_indices))

    print('Percent of noisy examples removed (should be close to 1, definitely greater than 1-|subset_fraction|):')
    print(1 - len(inter) / len(noisy_indices))

Files already downloaded and verified
fog
Files already downloaded and verified


RuntimeError: ignored

In [ ]:
# Get Subset using SAS
subset_dataset = SASSubsetDataset(
    dataset=cifar_noisy,
    subset_fraction=subset_fraction,
    num_downstream_classes=100,
    device=device,
    proxy_model=proxy_model,
    approx_latent_class_partition=partition,
    verbose=True
)

In [21]:
!wget https://zenodo.org/records/2535967/files/CIFAR-10-C.tar

--2023-12-15 05:20:19--  https://zenodo.org/records/2535967/files/CIFAR-10-C.tar
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2918471680 (2.7G) [application/octet-stream]
Saving to: ‘CIFAR-10-C.tar’

CIFAR-10-C.tar      100%[===================>]   2.72G  17.3MB/s    in 2m 48s  

2023-12-15 05:23:08 (16.6 MB/s) - ‘CIFAR-10-C.tar’ saved [2918471680/2918471680]



In [91]:
!ls -l CIFAR-10-C

total 2850132
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 brightness.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 contrast.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 defocus_blur.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 elastic_transform.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 fog.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 frost.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 gaussian_blur.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 gaussian_noise.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 glass_blur.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 impulse_noise.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 jpeg_compression.npy
-rw-rw-r-- 1 1189 2002     50128 Nov 25  2018 labels.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 motion_blur.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 pixelate.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 saturate.npy
-rw-rw-r-- 1 1189 2002 153600128 Nov 25  2018 shot_noise.npy
-rw-rw